In [1]:
from qtpg.team import Team
from qtpg.learner import Learner
from qtpg.program import Program
from qtpg.rule import Rule
from qtpg.figure13 import Figure13
from qtpg.figure13_hole_in_wall import Figure13HoleInWall
from qtpg.figure12 import Figure12
from qtpg.figure9 import Figure9
from qtpg.search_manager import SearchManager
import numpy as np
import uuid
import copy
import matplotlib.pyplot as plt

In [2]:
def check_overlap(state, region):
    if state[region[0]] == region[1] and region[2] <= state[not region[0]] <= region[3]:
        return True
    return False

def check_intersect(one, two):
    print(f'First region: {one}')
    print(f'Second region: {two}')
    firsts = []
    for i in range(one[2], one[3]+1):
        coord = [0, 0]
        coord[one[0]] = one[1]
        coord[not one[0]] = i
        firsts.append(coord)
    print('firsts---')
    print(firsts)
    seconds = []
    for i in range(two[2], two[3]+1):
        coord = [0, 0]
        coord[two[0]] = two[1]
        coord[not two[0]] = i
        seconds.append(coord)
    print('seconds---')
    print(seconds)

    pop_index = 0
    found = False
    
    for n in range(len(firsts)):
        for m in range(len(seconds)):
            if firsts[n] == seconds[m] and not found:
                print(f'popped! --> {firsts[n]} == {seconds[m]}')
                found = True
                pop_index = n
    
    if found:
        firsts.pop(pop_index)
    else:
        return []
    
    print('clipped firsts ----')
    print(firsts)
    
    print('new regions---')

    new_firsts = []

    region = [one[0], one[1], one[2], 0]
    for i in range(len(firsts)):
        if i > 0:
            before = firsts[i]
            if not firsts[i][not one[0]]-1 == firsts[i-1][not one[0]]:
                new_firsts.append(region)
                region = [one[0], one[1], i+1, i+1]
            else:
                region[3] = region[3]+1
    if len(new_firsts) == 0:
        new_firsts.append(region)

    print('firsts')
    for region in new_firsts:
        print(region)
    print('seconds')
    print(two)
    return new_firsts, two

In [3]:
# one = [1, 2, 0, 4]
# two = [0, 4, 0, 3]
one = [0, 0, 0, 4]
two = [0, 4, 0, 4]
check_intersect(one, two)

First region: [0, 0, 0, 4]
Second region: [0, 4, 0, 4]
firsts---
[[0, 0], [0, 1], [0, 2], [0, 3], [0, 4]]
seconds---
[[4, 0], [4, 1], [4, 2], [4, 3], [4, 4]]


[]

In [4]:
# env params
memorySize = 40
legalMove = 0.1
illegalMove = -0.01
outOfBounds = 0.0
memoryRepeat = -0.01
# memoryRepeat = 0.0
goalReached = 100

# envName = 'Figure 13'
# start_env = Figure13(5, 5, (2, 4), (0, 0), memorySize, legalMove, illegalMove, outOfBounds, memoryRepeat, goalReached)
# end_env = Figure13(5, 5, (0, 0), (2, 4), memorySize, legalMove, illegalMove, outOfBounds, memoryRepeat, goalReached)

# envName = 'Figure 13 Hole in Wall'
# start_env = Figure13HoleInWall(5, 5, (2, 4), (0, 0), memorySize, legalMove, illegalMove, outOfBounds, memoryRepeat, goalReached)
# end_env = Figure13HoleInWall(5, 5, (0, 0), (2, 4), memorySize, legalMove, illegalMove, outOfBounds, memoryRepeat, goalReached)

# envName = 'Figure 12'
# start_env = Figure12(5, 5, (4, 4), (0, 0), memorySize, legalMove, illegalMove, outOfBounds, memoryRepeat, goalReached)
# end_env = Figure12(5, 5, (0, 0), (4, 4), memorySize, legalMove, illegalMove, outOfBounds, memoryRepeat, goalReached)

envName = 'Figure 9'
start_env = Figure9(10, 10, (4, 9), (0, 3), memorySize, legalMove, illegalMove, outOfBounds, memoryRepeat, goalReached)
end_env = Figure9(10, 10, (0, 3), (4, 9), memorySize, legalMove, illegalMove, outOfBounds, memoryRepeat, goalReached)

In [5]:
maxTeamPool = 1 #5

# init everything for new run
startSearchManager = SearchManager(maxTeamPool)

adam_and_eve = []
start_env.reset()
init_team = Team(uuid.uuid4(), 0, 0, 1, 1, 0)
init_team.init_search(start_env, [2, 3])
adam_and_eve.append(init_team)
win = False
for adam in adam_and_eve:
    data = [adam, win]
    startSearchManager.evaluate_team(data)

endSearchManager = SearchManager(maxTeamPool)
    
adam_and_eve = []
end_env.reset()
init_team = Team(uuid.uuid4(), 0, 0, 1, 1, 0)
init_team.init_search(end_env, [0, 1])
adam_and_eve.append(init_team)
win = False
for adam in adam_and_eve:
    data = [adam, win]
    endSearchManager.evaluate_team(data)

intersect = False
gen = 0
while (not intersect):
    print('start_team')
    for parent in startSearchManager.teamPool:
        # only run if the team has not yet won
        if parent not in startSearchManager.winners:
#                 print(f'Parent: {parent.id}')
    #             print(f'Start: {parent.start_state}')
            start_env.current_state = parent.start_state

            # make offspring from parent, then let that offspring make a new rule
            child = copy.deepcopy(parent)
            child.id = uuid.uuid4()
            win = child.search_no_back_track(start_env)
            startSearchManager.evaluate_team([child, win])

    print('end team')
    for parent in endSearchManager.teamPool:
        # only run if the team has not yet won
        if parent not in endSearchManager.winners:
#                 print(f'Parent: {parent.id}')
    #             print(f'Start: {parent.start_state}')
            end_env.current_state = parent.start_state

            # make offspring from parent, then let that offspring make a new rule
            child = copy.deepcopy(parent)
            child.id = uuid.uuid4()
#             win = child.search_no_back_track(end_env)
            win = child.goal_start_search(end_env)
            endSearchManager.evaluate_team([child, win])
    
#     if startSearchManager.teamPool[0] overlaps with endSearchManager.teamPool[0].mostRecent.
    print(start_env.current_state)
    print(endSearchManager.teamPool[0].mostRecent.program.rule.region)
#     if check_overlap(start_env.current_state, endSearchManager.teamPool[0].mostRecent.program.rule.region):
#         overlap = True
#         print('overlap!')

    regions = check_intersect(
        startSearchManager.teamPool[0].mostRecent.program.rule.region, 
        endSearchManager.teamPool[0].mostRecent.program.rule.region
    )
    print('REGIONS')
    print(regions)
    if len(regions) > 0:
        print('HEYYY')
        intersect = True
        print('REGIONS!')
        print(regions[0][0])
        startSearchManager.teamPool[0].mostRecent.program.rule.region = regions[0][0]
        if len(regions[0]) > 1:
            print('regions 2:')
            print(regions[0][1])
            rule = copy.deepcopy(startSearchManager.teamPool[0].mostRecent.program.rule)
            rule.region = regions[0][1]
            learner = Learner(uuid.uuid4(), rule)
            startSearchManager.teamPool[0].learners.append(learner)
    gen += 1
    print(gen)
print('starts:')
for learner in startSearchManager.teamPool[0].learners:
    print(learner.program.rule.region)
                
print('ends:')
for learner in endSearchManager.teamPool[0].learners:
    print(learner.program.rule.region)

start_team
end team
[1, 9, 0, 4]
(0, 3)
[1, 9, 0, 4]
First region: [1, 3, 0, 3]
Second region: [1, 9, 0, 4]
firsts---
[[0, 3], [1, 3], [2, 3], [3, 3]]
seconds---
[[0, 9], [1, 9], [2, 9], [3, 9], [4, 9]]
REGIONS
[]
1
start_team
end team
[1, 9, 0, 4]
(0, 0)
[0, 1, 6, 8]
First region: [0, 0, 0, 4]
Second region: [0, 1, 6, 8]
firsts---
[[0, 0], [0, 1], [0, 2], [0, 3], [0, 4]]
seconds---
[[1, 6], [1, 7], [1, 8]]
REGIONS
[]
2
start_team
end team
[0, 1, 6, 8]
(0, 1)
[1, 8, 1, 1]
First region: [1, 1, 0, 9]
Second region: [1, 8, 1, 1]
firsts---
[[0, 1], [1, 1], [2, 1], [3, 1], [4, 1], [5, 1], [6, 1], [7, 1], [8, 1], [9, 1]]
seconds---
[[1, 8]]
REGIONS
[]
3
start_team
end team
(7, 0)
[0, 1, 7, 8]
First region: [0, 7, 0, 5]
Second region: [0, 1, 7, 8]
firsts---
[[7, 0], [7, 1], [7, 2], [7, 3], [7, 4], [7, 5]]
seconds---
[[1, 7], [1, 8]]
REGIONS
[]
4
start_team
end team
[0, 1, 7, 8]
(9, 2)
[1, 8, 1, 1]
First region: [1, 2, 5, 9]
Second region: [1, 8, 1, 1]
firsts---
[[5, 2], [6, 2], [7, 2], [8, 2]

In [6]:
first_team = startSearchManager.teamPool[0]
second_team = endSearchManager.teamPool[0]
first_team.learners.extend(second_team.learners)
team = first_team
for learner in team.learners:
    print(learner.program.rule.region)

[0, 0, 0, 2]
[0, 0, 4, 4]
[1, 3, 1, 3]
[0, 0, 0, 0]
[0, 0, 2, 4]
[1, 1, 0, 6]
[1, 1, 8, 9]
[0, 7, 0, 1]
[0, 7, 3, 5]
[1, 2, 5, 6]
[1, 2, 8, 9]
[0, 7, 0, 0]
[0, 7, 2, 5]
[1, 1, 0, 7]
[1, 1, 9, 9]
[0, 8, 0, 0]
[0, 8, 2, 5]
[1, 1, 0, 0]
[1, 9, 0, 4]
[0, 1, 6, 8]
[1, 8, 0, 0]
[1, 8, 1, 1]
[0, 1, 6, 7]
[0, 1, 7, 8]
[1, 8, 0, 0]
[1, 8, 1, 1]
[0, 1, 6, 7]
[0, 1, 7, 8]
[1, 7, 0, 0]
[1, 7, 1, 8]
[0, 5, 0, 6]
[1, 1, 0, 4]
[1, 1, 1, 8]


In [7]:
# for learner in team.learners:
#     print(learner.program.rule.region)
# team.learners[len(team.learners)-1].program.rule.region = [0, 0, 0, 1]

In [13]:
team.discount = 0.9
team.alpha = 0.5
# for learner in team.learners:
#     print(learner.program.rule.region)

# env = Figure13(5, 5, (2, 4), (0, 0), memorySize, legalMove, illegalMove, outOfBounds, memoryRepeat, goalReached)
env = Figure9(10, 10, (4, 9), (0, 3), memorySize, legalMove, illegalMove, outOfBounds, memoryRepeat, goalReached)

for i in range(env.rows):
#     print(f'Epoch: {i+1} --------')
    env.reset()
#     print(f'curr -> {env.current_state}')
    win = False
    selected_learner = team.select_learner(env, [0, 0, 0, 0])
    while not win:
        win, needs_update, reward, winning_action = team.q_evaluation(env, selected_learner)
#         print(reward)
#         print('hi')
        if not win:
            selected_learner = team.select_learner(env, selected_learner.program.rule.region)
            team.transition_update(reward, winning_action, needs_update, selected_learner)

    team.final_update(reward, winning_action, needs_update)

print(win)
for learner in team.learners:
    print(f'{learner.program.rule.region}')
    for i in range(len(learner.program.rule.action_set)):
          print(f'{learner.program.rule.action_set[i]} --> {learner.program.rule.value_set[i]}')

regions --> [0, 0, 0, 0] is eligible with [0, 0, 2, 4]
[0, 0, 2, 4] --> [2, 3]
we chose --> [0, 0, 2, 4]
[0, 0, 2, 4]
[2, 3]
[0, 0.11187500000000002]
3
(0, 2)
[0, 0, 2, 4]
[2, 3]
[0, 0.11187500000000002]
3
(0, 1)
Winning action: 3
regions --> [0, 0, 2, 4] is eligible with [0, 0, 0, 2]
regions --> [0, 0, 2, 4] is eligible with [1, 1, 0, 6]
regions --> [0, 0, 2, 4] is eligible with [1, 1, 0, 7]
regions --> [0, 0, 2, 4] is eligible with [1, 1, 0, 0]
regions --> [0, 0, 2, 4] is eligible with [1, 1, 0, 4]
[0, 0, 0, 2] --> [2, 3]
[1, 1, 0, 6] --> [0, 1]
[1, 1, 0, 7] --> [0, 1]
[1, 1, 0, 0] --> [0, 1]
[1, 1, 0, 4] --> [0, 1]
we chose --> [1, 1, 0, 7]
[1, 1, 0, 7]
[0, 1]
[0.39414119985744334, 0]
0
(1, 1)
[1, 1, 0, 7]
[0, 1]
[0.39414119985744334, 0]
0
(2, 1)
[1, 1, 0, 7]
[0, 1]
[0.39414119985744334, 0]
0
(3, 1)
[1, 1, 0, 7]
[0, 1]
[0.39414119985744334, 0]
0
(4, 1)
[1, 1, 0, 7]
[0, 1]
[0.39414119985744334, 0]
1
(3, 1)
[1, 1, 0, 7]
[0, 1]
[0.39414119985744334, 0]
0
(4, 1)
[1, 1, 0, 7]
[0, 1]
[0.3

regions --> [1, 1, 0, 4] is eligible with [1, 1, 0, 7]
regions --> [1, 1, 0, 4] is eligible with [0, 5, 0, 6]
regions --> [1, 1, 0, 4] is eligible with [1, 1, 1, 8]
[1, 1, 0, 6] --> [0, 1]
[1, 1, 0, 7] --> [0, 1]
[0, 5, 0, 6] --> [2, 3]
[1, 1, 1, 8] --> [0, 1]
we chose --> [1, 1, 1, 8]
[1, 1, 1, 8]
[0, 1]
[0, 0.40534632177628693]
1
(4, 1)
[1, 1, 1, 8]
[0, 1]
[0, 0.40534632177628693]
1
(3, 1)
[1, 1, 1, 8]
[0, 1]
[0, 0.40534632177628693]
1
(2, 1)
[1, 1, 1, 8]
[0, 1]
[0, 0.40534632177628693]
1
(1, 1)
[1, 1, 1, 8]
[0, 1]
[0, 0.40534632177628693]
1
(0, 1)
Winning action: 1
regions --> [1, 1, 1, 8] is eligible with [0, 0, 0, 2]
regions --> [1, 1, 1, 8] is eligible with [1, 1, 0, 6]
regions --> [1, 1, 1, 8] is eligible with [1, 1, 0, 7]
regions --> [1, 1, 1, 8] is eligible with [1, 1, 0, 0]
regions --> [1, 1, 1, 8] is eligible with [1, 1, 0, 4]
[0, 0, 0, 2] --> [2, 3]
[1, 1, 0, 6] --> [0, 1]
[1, 1, 0, 7] --> [0, 1]
[1, 1, 0, 0] --> [0, 1]
[1, 1, 0, 4] --> [0, 1]
we chose --> [1, 1, 0, 6]
[1, 

(1, 6)
value set is 0, action selected --> 3
[0, 1, 6, 7]
[2, 3]
[0, 0]
3
(1, 6)
value set is 0, action selected --> 3
[0, 1, 6, 7]
[2, 3]
[0, 0]
3
(1, 6)
value set is 0, action selected --> 2
[0, 1, 6, 7]
[2, 3]
[0, 0]
2
(1, 7)
value set is 0, action selected --> 3
[0, 1, 6, 7]
[2, 3]
[0, 0]
3
(1, 6)
value set is 0, action selected --> 2
[0, 1, 6, 7]
[2, 3]
[0, 0]
2
(1, 7)
value set is 0, action selected --> 2
[0, 1, 6, 7]
[2, 3]
[0, 0]
2
(1, 8)
Winning action: 2
regions --> [0, 1, 6, 7] is eligible with [0, 1, 6, 8]
regions --> [0, 1, 6, 7] is eligible with [1, 8, 1, 1]
regions --> [0, 1, 6, 7] is eligible with [0, 1, 7, 8]
regions --> [0, 1, 6, 7] is eligible with [1, 8, 1, 1]
regions --> [0, 1, 6, 7] is eligible with [0, 1, 7, 8]
[0, 1, 6, 8] --> [2, 3]
[1, 8, 1, 1] --> [0, 1]
[0, 1, 7, 8] --> [2, 3]
[1, 8, 1, 1] --> [0, 1]
[0, 1, 7, 8] --> [2, 3]
we chose --> [0, 1, 7, 8]
value set is 0, action selected --> 3
[0, 1, 7, 8]
[2, 3]
[0, 0]
3
(1, 7)
value set is 0, action selected --> 

(2, 1)
[1, 1, 0, 6]
[0, 1]
[0.5614301177990562, 0]
0
(3, 1)
[1, 1, 0, 6]
[0, 1]
[0.5614301177990562, 0]
1
(2, 1)
[1, 1, 0, 6]
[0, 1]
[0.5614301177990562, 0]
0
(3, 1)
[1, 1, 0, 6]
[0, 1]
[0.5614301177990562, 0]
0
(4, 1)
[1, 1, 0, 6]
[0, 1]
[0.5614301177990562, 0]
0
(5, 1)
[1, 1, 0, 6]
[0, 1]
[0.5614301177990562, 0]
0
(6, 1)
[1, 1, 0, 6]
[0, 1]
[0.5614301177990562, 0]
0
(7, 1)
Winning action: 0
regions --> [1, 1, 0, 6] is eligible with [0, 7, 0, 1]
regions --> [1, 1, 0, 6] is eligible with [1, 1, 0, 7]
regions --> [1, 1, 0, 6] is eligible with [1, 1, 1, 8]
[0, 7, 0, 1] --> [2, 3]
[1, 1, 0, 7] --> [0, 1]
[1, 1, 1, 8] --> [0, 1]
we chose --> [1, 1, 1, 8]
[1, 1, 1, 8]
[0, 1]
[0.20526510579831864, 0.496880733355687]
1
(6, 1)
[1, 1, 1, 8]
[0, 1]
[0.20526510579831864, 0.496880733355687]
1
(5, 1)
[1, 1, 1, 8]
[0, 1]
[0.20526510579831864, 0.496880733355687]
1
(4, 1)
[1, 1, 1, 8]
[0, 1]
[0.20526510579831864, 0.496880733355687]
1
(3, 1)
[1, 1, 1, 8]
[0, 1]
[0.20526510579831864, 0.496880733355687]


[1, 1, 0, 7]
[0, 1]
[0.6205901385821095, 0]
0
(4, 1)
[1, 1, 0, 7]
[0, 1]
[0.6205901385821095, 0]
0
(5, 1)
[1, 1, 0, 7]
[0, 1]
[0.6205901385821095, 0]
0
(6, 1)
[1, 1, 0, 7]
[0, 1]
[0.6205901385821095, 0]
0
(7, 1)
[1, 1, 0, 7]
[0, 1]
[0.6205901385821095, 0]
0
(8, 1)
Winning action: 0
regions --> [1, 1, 0, 7] is eligible with [1, 1, 8, 9]
regions --> [1, 1, 0, 7] is eligible with [1, 1, 1, 8]
[1, 1, 8, 9] --> [0, 1]
[1, 1, 1, 8] --> [0, 1]
we chose --> [1, 1, 1, 8]
[1, 1, 1, 8]
[0, 1]
[0.20526510579831864, 0.5746144590140019]
1
(7, 1)
[1, 1, 1, 8]
[0, 1]
[0.20526510579831864, 0.5746144590140019]
1
(6, 1)
[1, 1, 1, 8]
[0, 1]
[0.20526510579831864, 0.5746144590140019]
1
(5, 1)
[1, 1, 1, 8]
[0, 1]
[0.20526510579831864, 0.5746144590140019]
1
(4, 1)
[1, 1, 1, 8]
[0, 1]
[0.20526510579831864, 0.5746144590140019]
1
(3, 1)
[1, 1, 1, 8]
[0, 1]
[0.20526510579831864, 0.5746144590140019]
1
(2, 1)
[1, 1, 1, 8]
[0, 1]
[0.20526510579831864, 0.5746144590140019]
1
(1, 1)
[1, 1, 1, 8]
[0, 1]
[0.2052651057983

regions --> [0, 0, 2, 4] is eligible with [0, 0, 0, 2]
regions --> [0, 0, 2, 4] is eligible with [1, 1, 0, 6]
regions --> [0, 0, 2, 4] is eligible with [1, 1, 0, 7]
regions --> [0, 0, 2, 4] is eligible with [1, 1, 0, 0]
regions --> [0, 0, 2, 4] is eligible with [1, 1, 0, 4]
[0, 0, 0, 2] --> [2, 3]
[1, 1, 0, 6] --> [0, 1]
[1, 1, 0, 7] --> [0, 1]
[1, 1, 0, 0] --> [0, 1]
[1, 1, 0, 4] --> [0, 1]
we chose --> [1, 1, 0, 0]
[1, 1, 0, 0]
[0, 1]
[0.6294917301200982, 0]
0
(1, 1)
Winning action: 0
regions --> [1, 1, 0, 0] is eligible with [1, 1, 0, 6]
regions --> [1, 1, 0, 0] is eligible with [1, 1, 0, 7]
regions --> [1, 1, 0, 0] is eligible with [1, 1, 0, 4]
regions --> [1, 1, 0, 0] is eligible with [1, 1, 1, 8]
[1, 1, 0, 6] --> [0, 1]
[1, 1, 0, 7] --> [0, 1]
[1, 1, 0, 4] --> [0, 1]
[1, 1, 1, 8] --> [0, 1]
we chose --> [1, 1, 0, 4]
[1, 1, 0, 4]
[0, 1]
[0.8973931131452606, 0]
0
(2, 1)
[1, 1, 0, 4]
[0, 1]
[0.8973931131452606, 0]
0
(3, 1)
[1, 1, 0, 4]
[0, 1]
[0.8973931131452606, 0]
0
(4, 1)
[1, 1, 

(1, 8)
Winning action: 2
regions --> [0, 1, 6, 7] is eligible with [0, 1, 6, 8]
regions --> [0, 1, 6, 7] is eligible with [1, 8, 1, 1]
regions --> [0, 1, 6, 7] is eligible with [0, 1, 7, 8]
regions --> [0, 1, 6, 7] is eligible with [1, 8, 1, 1]
regions --> [0, 1, 6, 7] is eligible with [0, 1, 7, 8]
[0, 1, 6, 8] --> [2, 3]
[1, 8, 1, 1] --> [0, 1]
[0, 1, 7, 8] --> [2, 3]
[1, 8, 1, 1] --> [0, 1]
[0, 1, 7, 8] --> [2, 3]
we chose --> [1, 8, 1, 1]
[1, 8, 1, 1]
[0, 1]
[0, 0.06525]
1
(0, 8)
Winning action: 1
regions --> [1, 8, 1, 1] is eligible with [1, 8, 0, 0]
regions --> [1, 8, 1, 1] is eligible with [1, 8, 0, 0]
[1, 8, 0, 0] --> [0, 1]
[1, 8, 0, 0] --> [0, 1]
we chose --> [1, 8, 0, 0]
[1, 8, 0, 0]
[0, 1]
[0.09, 0]
0
(1, 8)
Winning action: 0
regions --> [1, 8, 0, 0] is eligible with [0, 1, 6, 8]
regions --> [1, 8, 0, 0] is eligible with [1, 8, 1, 1]
regions --> [1, 8, 0, 0] is eligible with [0, 1, 7, 8]
regions --> [1, 8, 0, 0] is eligible with [1, 8, 1, 1]
regions --> [1, 8, 0, 0] is eligi

[0, 1]
[1.0011520711848525, 0]
0
(1, 1)
[1, 1, 0, 7]
[0, 1]
[1.0011520711848525, 0]
0
(2, 1)
[1, 1, 0, 7]
[0, 1]
[1.0011520711848525, 0]
0
(3, 1)
[1, 1, 0, 7]
[0, 1]
[1.0011520711848525, 0]
0
(4, 1)
[1, 1, 0, 7]
[0, 1]
[1.0011520711848525, 0]
0
(5, 1)
[1, 1, 0, 7]
[0, 1]
[1.0011520711848525, 0]
0
(6, 1)
[1, 1, 0, 7]
[0, 1]
[1.0011520711848525, 0]
0
(7, 1)
[1, 1, 0, 7]
[0, 1]
[1.0011520711848525, 0]
0
(8, 1)
Winning action: 0
regions --> [1, 1, 0, 7] is eligible with [1, 1, 8, 9]
regions --> [1, 1, 0, 7] is eligible with [1, 1, 1, 8]
[1, 1, 8, 9] --> [0, 1]
[1, 1, 1, 8] --> [0, 1]
we chose --> [1, 1, 8, 9]
[1, 1, 8, 9]
[0, 1]
[0, 1.1591615173468983]
1
(7, 1)
Winning action: 1
regions --> [1, 1, 8, 9] is eligible with [0, 7, 0, 1]
regions --> [1, 1, 8, 9] is eligible with [1, 1, 0, 7]
regions --> [1, 1, 8, 9] is eligible with [1, 1, 1, 8]
[0, 7, 0, 1] --> [2, 3]
[1, 1, 0, 7] --> [0, 1]
[1, 1, 1, 8] --> [0, 1]
we chose --> [0, 7, 0, 1]
[0, 7, 0, 1]
[2, 3]
[0.8431602770390468, 0]
2
(7, 2)


In [14]:
region_action_scores = []
for learner in team.learners:
    if learner.program.rule.value_set[0] > learner.program.rule.value_set[1]:
        region_action_scores.append({'region': learner.program.rule.region, 'action': learner.program.rule.action_set[0], 'q': learner.program.rule.value_set[0]})
    else:
        region_action_scores.append({'region': learner.program.rule.region, 'action': learner.program.rule.action_set[1], 'q': learner.program.rule.value_set[1]})

sorted_region_action_scores = sorted(region_action_scores, key=lambda value: float(value['q']), reverse=True)
print(sorted_region_action_scores)
for record in sorted_region_action_scores:
    print(record)
action_states = []
for record in sorted_region_action_scores:
    action = ''
    if record['action'] == 0:
        action = '\u2191'
    elif record['action'] == 1:
        action = '\u2193'
    elif record['action'] == 2:
        action = '\u2192'
    elif record['action'] == 3:
        action = '\u2190'
    
    for i in range(record['region'][3] - record['region'][2]+1):
        state = [0, 0]
        state[record['region'][0]] = record['region'][1]
        state[not record['region'][0]] = record['region'][2] + i
        
        # ensure we do not enter duplicate states
        found = 0
        for pair in action_states:
            if pair['state'] == (state[0], state[1]):
                found = 1
                
        if found == 0:
            action_states.append({'state': (state[0], state[1]), 'action': action})

# add in the rest of the states, either they are not visited, or illegal
for n in reversed(range(env.rows)):
    for m in range(env.cols):
        action = ''
        found = 0
        for record in action_states:
            if record['state'] == (n, m):
                found = 1
                action = record['action']
        if found == 0:
            if not env.check_legal((n, m)):
                action = 'X'
            else:
                action = '?'
        print(f'{action} ', end='')
    print('\n')

# print(action_states)

[{'region': [1, 9, 0, 4], 'action': 0, 'q': 100.00224609374999}, {'region': [0, 1, 7, 8], 'action': 2, 'q': 88.43003906249999}, {'region': [0, 1, 7, 8], 'action': 2, 'q': 60.604218749999994}, {'region': [1, 8, 0, 0], 'action': 0, 'q': 58.04573730468749}, {'region': [1, 7, 0, 0], 'action': 0, 'q': 28.379237588745116}, {'region': [1, 7, 1, 8], 'action': 1, 'q': 28.260065703759768}, {'region': [0, 5, 0, 6], 'action': 2, 'q': 17.852858144777834}, {'region': [1, 8, 1, 1], 'action': 1, 'q': 17.1395818359375}, {'region': [0, 1, 6, 7], 'action': 2, 'q': 11.822340429687499}, {'region': [1, 1, 0, 4], 'action': 0, 'q': 7.337088199368755}, {'region': [1, 1, 1, 8], 'action': 1, 'q': 1.986281786967889}, {'region': [0, 7, 2, 5], 'action': 3, 'q': 0.9931637981437964}, {'region': [1, 1, 0, 7], 'action': 0, 'q': 0.9692248925891407}, {'region': [0, 7, 0, 1], 'action': 2, 'q': 0.93791794642229}, {'region': [1, 1, 8, 9], 'action': 1, 'q': 0.9370069718966628}, {'region': [0, 0, 2, 4], 'action': 3, 'q': 0.91